# Contando N-Gramas

In [2]:
from nltk.corpus import gutenberg
sents = list(gutenberg.sents('austen-emma.txt'))

Primero vemos cómo imprimir todos los trigramas de una sola oración:

In [3]:
sent = sents[0]

n = 3  # trigramas
for i in range(len(sent) - n + 1):
    print(sent[i:i+n])

['[', 'Emma', 'by']
['Emma', 'by', 'Jane']
['by', 'Jane', 'Austen']
['Jane', 'Austen', '1816']
['Austen', '1816', ']']


Ahora veamos cómo contar los trigramas de todas las oraciones:

In [4]:
from collections import defaultdict

count = defaultdict(int)

for sent in sents:
    for i in range(len(sent) - n + 1):
        ngram = tuple(sent[i:i+n])  # los diccionarios no pueden guardar listas, pero sí tuplas
        count[ngram] += 1

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize


def split_and_tokenize(text):
    tokens = []
    for sent in sent_tokenize(text):
        tokens.append(word_tokenize(sent))
    return tokens


split_and_tokenize("Hay que dejar de robar por dos años. Estamos bien los 33")

[['Hay', 'que', 'dejar', 'de', 'robar', 'por', 'dos', 'años', '.'],
 ['Estamos', 'bien', 'los', '33']]

In [26]:
def _generate_n_grams_for_sentence(sentence, n):
    ngrams = []
    
    """
    Los n-1 primeros tokens tengo que rellenarlos 
    """
    m = len(sentence)
    
    for i in range(min(n-1, m)):
        ngram = ['<s>'] * (n-(i+1)) + sentence[0:i+1]
        ngrams.append(ngram)
        
    for i in range(n-2, m-n+1):
        ngrams.append(sentence[i:i+n])
    
    for i in range(m-n+1, m):
        ngram = sentence[i:m] + ['</s>'] * (n - (m-i))
        ngrams.append(ngram)
        
    return ngrams
        
def generate_ngrams(sents, n):
    """
    Generar n-gramas a partir de las sentencias
    """
    ngrams = []
    
    for sent in sents:
        ngrams += _generate_n_grams_for_sentence(sent, n)
    return ngrams

texto = "Sí se puede! Sí se puede!"
sents = split_and_tokenize(texto)
#['<s>'] * 10
print(sents)
len(generate_ngrams(sents, 0))

[['Sí', 'se', 'puede', '!'], ['Sí', 'se', 'puede', '!']]


14

In [15]:
texto = "El Bajo porteño ha sido protagonista de constantes cambios a lo largo de la historia\
 y parece que en los próximos años las obras continuarán. Polémicos traslados de estatuas, reacomodo\
 de calles, estacionamientos, nuevos edificios y enrejado son algunas de las modificaciones que realizaron\
 los distintos gobiernos desde 1937, cuando toda el área era de acceso público."

sents = split_and_tokenize(texto)
ngrams = generate_ngrams(sents, 2)

In [16]:
count = defaultdict(int)

for ngram in ngrams:
    count[tuple(ngram)] += 1
    
count

defaultdict(int,
            {(',', 'cuando'): 1,
             (',', 'estacionamientos'): 1,
             (',', 'nuevos'): 1,
             (',', 'reacomodo'): 1,
             ('1937', ','): 1,
             ('<s>', 'El'): 1,
             ('<s>', 'Polémicos'): 1,
             ('Bajo', 'porteño'): 1,
             ('El', 'Bajo'): 1,
             ('Polémicos', 'traslados'): 1,
             ('a', 'lo'): 1,
             ('acceso', 'público'): 1,
             ('algunas', 'de'): 1,
             ('años', 'las'): 1,
             ('calles', ','): 1,
             ('cambios', 'a'): 1,
             ('constantes', 'cambios'): 1,
             ('continuarán', '.'): 1,
             ('cuando', 'toda'): 1,
             ('de', 'acceso'): 1,
             ('de', 'calles'): 1,
             ('de', 'constantes'): 1,
             ('de', 'estatuas'): 1,
             ('de', 'la'): 1,
             ('de', 'las'): 1,
             ('desde', '1937'): 1,
             ('distintos', 'gobiernos'): 1,
             ('edificio

El código sirve para n-gramas en general.

Tareas pendientes:
- Agregar marcadores de principio y final de oración
- Contar n-gramas y (n-1)-gramas al mismo tiempo.

# Generando Lenguaje Natural

El siguiente modelo de bigramas se aprende a partir de dos oraciones:
- "el gato come pescado"
- "la gata come salmón"


In [78]:
probs = {
    '<s>': {'el': 0.5, 'la': 0.5},
    # '<s>': {'el': 0.6, 'la': 0.2, 'los': 0.1, 'las': 0.1},
    'el': {'gato': 1.0},
    'gato': {'come': 1.0},
    'come': {'pescado': 0.5, 'salmón': 0.5},
    'pescado': {'.': 1.0},
    '.': {'</s>': 1.0},
    'la': {'gata': 1.0},
    'gata': {'come': 1.0},
    'salmón': {'.': 1.0},
}

list(probs['<s>'].items())  # convertir un diccionario a lista de pares

[('la', 0.5), ('el', 0.5)]

Cada entrada del diccionario contiene una distribución discreta finita para la palabra siguiente dada la palabra anterior. Samplear de una distribución discreta finita es tan fácil como samplear un número al azar entre 0 y 1 y ver en qué región cae (ver [Wikipedia](https://en.wikipedia.org/wiki/Pseudo-random_number_sampling#Finite_discrete_distributions)).

Empezamos sampleando la primer palabra:

In [88]:
from random import random

def sample(problist):
    r = random()  # entre 0 y 1
    i = 0
    word, prob = problist[0]
    acum = prob
    while r > acum:
        i += 1
        word, prob = problist[i]
        acum += prob
    
    return word

sample(list(probs['<s>'].items()))

'la'

Podemos ver que el resultado del sampleo se corresponde con las probabilidades:

In [90]:
results = [sample(list(probs['<s>'].items())) for i in range(1000)]

from collections import Counter
print(Counter(results))

Counter({'el': 505, 'la': 495})


**Observaciones:**
- Si se ordena la lista de probabilidades de mayor a menor, el sampling es más rápido.
- El sampling también se puede hacer usando [random.choices](https://docs.python.org/3/library/random.html#random.choices) de python
ó [random.choice](https://stackoverflow.com/questions/11373192/generating-discrete-random-variables-with-specified-weights-using-scipy-or-numpy) de numpy.

Ahora veamos cómo samplear una oración completa:

In [112]:
word = '<s>'
while word != '</s>':
    problist = list(probs[word].items())
    word = sample(problist)
    print(word)

la
gata
come
pescado
.
</s>


Acá se ve que se pueden generar oraciones nuevas (no vistas en tiempo de entrenamiento).

Tareas pendientes:
- adaptar el código a n-gramas en general: usar tuplas como claves en probs!
- precalcular las listas ordenadas de mayor a menor (ver sorted_prob en los tests)